# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## 1. Изучим общую информацию

In [1]:
# Импортируем данные и получим общую информацию о датасете
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [2]:
# Выведем первые 10 строк датасета
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Теперь проверим все столбцы на предмет дубликатов и пропусков.

In [3]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Здесь есть ошибки: 20 и -1 не могут быть такие данные о детях. Скорее всего это опечатки.

In [4]:
data['days_employed'].isnull().sum()

2174

2174 пропуска в данных. Возможно не были внесены или загружены данные о месте работы и заработке.

In [5]:
data['dob_years'].value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

Здесь есть возраст "0" в количестве 101. Возможно не были заполнены данные о возврасте.

In [6]:
data['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

Здесь задублированы данные с разным регистром. Данные вероятно вводились вручную.

In [7]:
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

Здесь можно исправить "Не женат" на "не женат".

In [8]:
data['family_status_id'].value_counts()

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

Тут все ок.

In [9]:
data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Здесь есть одно значение XNA - не указан пол заявителя.

In [10]:
data['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64

Здесь все ок.

In [11]:
data['debt'].value_counts()

0    19784
1     1741
Name: debt, dtype: int64

Здесь ок. На заметку - 1741 не платят в срок.

In [12]:
data['total_income'].isnull().sum()

2174

Здесь 2174 пропусков.

In [13]:
data['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

Здесь задулированы данные, тк был свободный ввод текста. Всего получается только четыре типа целей для кредита: свадьба, недвижимость, автомобиль и образование.

Проверим еще на наличие дубликатов:

In [14]:
data.duplicated().sum()

54

В данных 54 дубликата.

**Вывод**

Из общей информации о датасете следует отметить отсутствующие значения в столбцах days_employed и total_income: их число 19351 не соответствует общему числу записей 21525.

Из первых 10 строк датасета следует выделить следующие наблюдения:
* в столбце children есть ошибки: 20 и -1 детей;
* столбцы days_employed и total_income имеют пропуски;
* столбец days_employed имеет отрицательные значения;
* в столбце dob_years есть возраст 0;
* в столбце education проблема с регистром данных;
* в столбце gender есть одно значение XNA;
* в столбце purpose задублированы данные, которые можно конвертировать в 4 категории;
* float значения необходимо заменить на integer;
* в данных 54 дубликата.

## 2. Предобработка данных

### Обработка пропусков

В данных по детям есть две ошибки: -1 и 20

In [15]:
# Уберем отрицательные значения в данных и заменим 20 на 2 в days_employed, вернув значения в формат int:
data['children'] = list(map(abs, data['children']))
data.loc[data['children'] == 20, 'children'] = 2
data['children'] = data['children'].astype(int)

В столбце gender есть одно значение XNA.

In [17]:
# Заменим XNA на F, которых больше всего в выборке:
data.loc[data['gender'] == 'XNA', 'gender'] = 'F'

В соответствии с общей информацией о данных в столбце total_income есть пропуски данных. Такие данные можно было запросить, тк выдача кредита не предусмотрена без указания дохода заемщика, однако такой возможности нет. Предлагается заполнить пропуски в столбце total_income медианным значением в выборке.

In [18]:
# Уберем отрицательные значения в данных и проверим значения days_employed на выбросы:
data['days_employed'] = list(map(abs, data['days_employed']))
data['days_employed'].max()

401755.40047533

В данных days_employed есть выбросы - не может быть 401 755 дней.

In [19]:
# Заполним пропуски в столбце days_employed медианным значением:
days_employed_median = data['days_employed'].median()
data['days_employed'] = data['days_employed'].fillna(value = days_employed_median)

Данные переведены в положительные значения, пропуски в столбце days_employed заполнены. Выбрана медиана, тк есть выбросы в данных.

Для столбца total_income можно было бы использовать медиану, однако это может повлиять на распределение, например появятся всплески на гистограмме, поэтому предлагается выполнить классификацию по трем характеристикам: income_type и education,  и затем заменить пропуски медианами по выбранным характеристикам.

In [205]:
# Рассчитаем медианное значение столбца total_income с группировкой по столбцам income_type и education:
income_median = data.groupby(['income_type', 'education_id'])['total_income'].median()
income_median

income_type      education_id
безработный      0               202722.511368
                 1                59956.991984
в декрете        1                53829.130729
госслужащий      0               172511.107016
                 1               136652.970357
                 2               160592.345303
                 3               148339.290825
                 4               111392.231107
компаньон        0               201785.400018
                 1               159070.690289
                 2               179867.152890
                 3               136798.905143
пенсионер        0               144240.768611
                 1               114842.854099
                 2               120136.896353
                 3               102598.653164
                 4               177088.845999
предприниматель  0               499163.144947
сотрудник        0               165640.744634
                 1               136555.108821
                 2            

In [206]:
# Создадим функцию для замены пропусков в столбце total_income на значения из выборки выше:
def func(row):
    if pd.isna(row['total_income']):
        return income_median.loc[row['income_type'], row['education_id']]
    return row['total_income']

# Применим записанную фукнцию:
data['total_income'] = data.apply(func, axis = 1)

Пропуски в столбце total_income заполнены.

**Вывод**

In [207]:
data.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [208]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [209]:
data['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [210]:
data['gender'].value_counts()

F    14237
M     7288
Name: gender, dtype: int64

Пропусков больше нет, отрицательных значений и ошибок тоже.

### Замена типа данных

In [211]:
# Заменим в столбцах тип данных с float на int
data['days_employed'] = data['days_employed'].astype(int)
data['total_income'] = data['total_income'].astype(int)

**Вывод**

In [212]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


Данные в столбцах days_employed и total_income изменены на int.

### Обработка дубликатов

In [213]:
# Поменяем регистр данных в столбцах education и family_status и уберем дубликаты в данных:
data['education'] = data['education'].str.lower()
data['family_status'] = data['family_status'].str.lower()

**Вывод**

In [214]:
data['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [215]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


Теперь в данных нет дубликатов.

### Лемматизация

Теперь нужно провести лемматизацию столбца purpose:

In [216]:
# Импортируем pymystem3, выделим значения в столбце purpose в строку и лемматизируем ее:
from pymystem3 import Mystem
m = Mystem()

text = data['purpose'].unique()
text = " ".join(text)

lemmas = m.lemmatize(text)

from collections import Counter
Counter(lemmas)

Counter({'покупка': 10,
         ' ': 96,
         'жилье': 7,
         'приобретение': 1,
         'автомобиль': 9,
         'дополнительный': 2,
         'образование': 9,
         'сыграть': 1,
         'свадьба': 3,
         'операция': 4,
         'с': 5,
         'на': 4,
         'проведение': 1,
         'для': 2,
         'семья': 1,
         'недвижимость': 10,
         'коммерческий': 2,
         'жилой': 2,
         'строительство': 3,
         'собственный': 1,
         'подержать': 2,
         'свой': 4,
         'со': 1,
         'заниматься': 2,
         'сделка': 2,
         'получение': 3,
         'высокий': 3,
         'профильный': 1,
         'сдача': 1,
         'ремонт': 1,
         '\n': 1})

Максимальные значения имеют: жилье, автомобиль, образование, свадьба, операция, недвижимость, строительство.

In [217]:
# Проведем стемминг слов и выделим их основу:
from nltk.stem import SnowballStemmer 
russian_stemmer = SnowballStemmer('russian')  

purpose = ['жилье', 'недвижимость', 'автомобиль', 'свадьба', 'образование', 'строительство']
for word in purpose:
    print('Исходное слово - ' + word + ', после стемминга - ' + russian_stemmer.stem(word))

Исходное слово - жилье, после стемминга - жил
Исходное слово - недвижимость, после стемминга - недвижим
Исходное слово - автомобиль, после стемминга - автомобил
Исходное слово - свадьба, после стемминга - свадьб
Исходное слово - образование, после стемминга - образован
Исходное слово - строительство, после стемминга - строительств


**Вывод**

Получили основные цели кредита: жилье, автомобиль, образование, свадьба, недвижимость, строительство и их основы: жил, недвижим, автомобил, свадьб, образован, строительств.

### Категоризация данных

Проведем категоризацию данных в столбце purpose:

In [218]:
# Создадим функцию для категоризации данных:
def change_purpose(word_purpose):
    if 'жил'in word_purpose or 'недвижим' in word_purpose or 'строительств' in word_purpose:
        return 'недвижимость'
    if 'автомобил' in word_purpose:
        return 'автомобиль'
    if 'свадьб' in word_purpose:
        return 'свадьба'
    if 'образование' in word_purpose:
        return 'образование'

data['purpose_categories'] = data['purpose'].apply(change_purpose)
data['purpose_categories'].value_counts()

недвижимость    10840
автомобиль       4315
образование      2706
свадьба          2348
Name: purpose_categories, dtype: int64

**Вывод**

Добавлен столбец с новой более понятной категоризацией на 4 вида целей получения кредита.

## 3. Исследование данных

1. Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [219]:
# Составим сводную таблицу из данных двух столбцов children и debt и добавим столбец с долевым значением:
children_pivot = data.pivot_table(index = 'children', values = 'debt', aggfunc = ['sum', 'count'])
children_pivot['share'] = children_pivot['sum'] / children_pivot['count']
children_pivot.sort_values(by='share')

,sum,count,share
,debt,debt,
children,,,
5,0,9,0.000000
0,1063,14149,0.075129
3,27,330,0.081818
1,445,4865,0.091470
2,202,2131,0.094791
4,4,41,0.097561


**Вывод**

В полученных данных можно отметить тренд на рост доли невовзрата кредитов, чем больше у заемщиков детей. Наибольшее число невозвратов имеют заемщики с 4мя детьми - 9,75%. Наименьшая вероятность невозврата кредита у тех, у кого нет детей - 7,51%.

2. Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [220]:
# Составим сводную таблицу из данных двух столбцов family_status и debt:
family_pivot = data.pivot_table(index = 'family_status', values = 'debt', aggfunc = ['sum', 'count'])
family_pivot['share'] = family_pivot['sum'] / family_pivot['count']
family_pivot.sort_values(by='share')

,sum,count,share
,debt,debt,
family_status,,,
вдовец / вдова,63,960,0.065625
в разводе,85,1195,0.071130
женат / замужем,931,12380,0.075202
гражданский брак,388,4177,0.092890
не женат / не замужем,274,2813,0.097405


**Вывод**

Наибольшая вероятность невозврата кредита имеют группы "не женат/не замужем" - 9,74% и "гражданский брак" - 9,29%. Наименьшая вероятность невовзрата кредитов у тех, кто в разводе 7,11% или вдовец/вдова 6,56%.

3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [221]:
# Для анализа доходов необходимо их распределить по категориям:
def income_group(income):
    if income < 30000:
        return 'бедные'
    if income < 80000:
        return 'нижний средний'
    if income < 200000:
        return 'средний класс'
    if income < 400000:
        return 'верхний средний'
    return 'богатые'

data['income_group'] = data['total_income'].apply(income_group)
data['income_group'].value_counts()

средний класс      13994
верхний средний     4726
нижний средний      2254
богатые              529
бедные                22
Name: income_group, dtype: int64

In [222]:
# Теперь посчитаем доли невозвратов по столбцу debt:
income_pivot = data.pivot_table(index = 'income_group', values = 'debt', aggfunc = ['sum', 'count'])
income_pivot['share'] = income_pivot['sum'] / income_pivot['count']
income_pivot.sort_values(by='share')

,sum,count,share
,debt,debt,
income_group,,,
богатые,31,529,0.058601
верхний средний,335,4726,0.070884
нижний средний,172,2254,0.076309
средний класс,1201,13994,0.085822
бедные,2,22,0.090909


**Вывод**

Богатые платят вовремя, доля невозвратов - 5,86%, хуже всего справляются бедные - 9,09%.

4. Как разные цели кредита влияют на его возврат в срок?

In [223]:
# Составим сводную таблицу из данных двух столбцов purpose_categories и debt:
purpose_pivot = data.pivot_table(index = 'purpose_categories', values = 'debt', aggfunc = ['sum', 'count'])
purpose_pivot['share'] = purpose_pivot['sum'] / purpose_pivot['count']
purpose_pivot.sort_values(by='share')

,sum,count,share
,debt,debt,
purpose_categories,,,
недвижимость,782,10840,0.072140
свадьба,186,2348,0.079216
образование,236,2706,0.087214
автомобиль,403,4315,0.093395


**Вывод**

Наибольшая вероятность невозврата кредита на покупку автомобиля - 9,34%, наименьшая вероятность у тех, кто берет кредит на операции с недвижимостью - 7,21%.

## Общий вывод

С целью создания кредитного скоринга и на основании представленных данных можно сделать следующие выводы:
* самые надежные клиенты это те, у кого нет детей (7,51%), а самые ненадежные те, у кого 4 ребенка (9,76%).
* самые надежные клиенты это те, кто вдовец/вдова (6,56%), а самые ненадежные те, кто не женат / не замужем (9,74%).
* самые надежные клиенты - богатые (5,86%), а самые ненадежные - бедные (9,09%).
* самые надежные клиенты это те, кто берет кредит на операции с недвижимостью (7,21%), а самые ненадежные те, кто обращается в банк с целью получить деньги на покупку автомобиля (9,33%).